In [1]:
from tree_sitter import Language, Parser

Language.build_library(
    ".local/build/json_prefix.so",
    [
        "tree-sitter-json-prefix"
    ]
)
JSON_LANGUAGE = Language(".local/build/json_prefix.so", "json_prefix")

parser = Parser()
parser.set_language(JSON_LANGUAGE)

In [2]:
def print_tree(node, depth=0):
    node_type = node.type
    has_error = node.has_error
    padding = " " * depth
    is_missing = node.is_missing
    print(f'{padding}<type="{node_type}" has_error=({has_error}) is_missing=({is_missing}) range=({node.start_byte}, {node.end_byte})>')
    for child in node.children:
        print_tree(child, depth + 1)
    print(f"{padding}</type={node_type}>")

def print_tree_from_string(s):
    tree = parser.parse(bytes(s, "utf8"))
    print_tree(tree.root_node)

In [3]:
def depth_first_traversal(node):
    """Depth-first traversal of the tree"""
    yield node
    for child in node.children:
        yield from depth_first_traversal(child)

def is_json_or_prefix(s):
    tree = parser.parse(bytes(s, "utf8"))
    return not any(node.has_error for node in depth_first_traversal(tree.root_node))

s = '{\n"a'

print(is_json_or_prefix(s))
print_tree_from_string(s)
tree = parser.parse(bytes(s, "utf8"))
print(tree.root_node.sexp())

True
<type="document" has_error=(False) is_missing=(False) range=(0, 4)>
 <type="prefix_object" has_error=(False) is_missing=(False) range=(0, 4)>
  <type="{" has_error=(False) is_missing=(False) range=(0, 1)>
  </type={>
  <type="prefix_pair" has_error=(False) is_missing=(False) range=(2, 4)>
   <type="prefix_string" has_error=(False) is_missing=(False) range=(2, 4)>
    <type=""" has_error=(False) is_missing=(False) range=(2, 3)>
    </type=">
    <type="string_content" has_error=(False) is_missing=(False) range=(3, 4)>
    </type=string_content>
   </type=prefix_string>
  </type=prefix_pair>
 </type=prefix_object>
</type=document>
(document (prefix_object (prefix_pair key: (prefix_string (string_content)))))


In [4]:
parser.parse(b"[1, 2, 3]").root_node.sexp()

'(document (array (number) (number) (number)))'

In [5]:
parser.parse(b"[1, 2, 3").root_node.sexp()

'(document (prefix_array (number) (number) (number)))'

In [6]:
parser.parse(b"[1, 2, ]").root_node.sexp()

'(document (array (number) (number) (MISSING number)))'

In [7]:
parser.parse(b"1e1").root_node.sexp()

'(document (number))'

In [8]:
parser.parse(b"1e").root_node.sexp()

'(document (prefix_number))'

In [9]:
parser.parse(b"1a").root_node.sexp()

"(document (number) (ERROR (UNEXPECTED 'a')))"